<a href="https://colab.research.google.com/github/ajayostawal/Resume-Parsing/blob/main/resume_info_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from zipfile import ZipFile
file_name = '/content/resume2.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [38]:
pip install pdfminer


In [46]:
import spacy #nlp
import pdfminer #pdf2tx
import re #regex
import os #file manip
import pandas as pd #csv - tabular

In [5]:
pip install pdfminer.six


     |████████████████████████████████| 5.6 MB 22.1 MB/s 
     |████████████████████████████████| 3.6 MB 49.4 MB/s 


In [6]:
pip install pdfminer2


     |████████████████████████████████| 117 kB 24.2 MB/s 


In [7]:
pip install pdfminer3

     |████████████████████████████████| 5.0 MB 21.3 MB/s 
  Created wheel for pdfminer3: filename=pdfminer3-2018.12.3.0-py3-none-any.whl size=117822 sha256=f3450498b6f0edeab1a65a84b64ce9b58a84ac7c633906ee80bee67992eb4ba5
  Stored in directory: /root/.cache/pip/wheels/f6/1b/21/339d1825e274c4a9829233a986f93dcedb98913f98e85b2916
Successfully built pdfminer3


In [47]:
from resume2 import pdf2txt

In [48]:
def convert_pdf(f):

    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt"
    output_filepath = os.path.join("/content/resume2/output/txt/", output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath]) #pdf to txt and save it in the given location
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

In [49]:
nlp = spacy.load("en_core_web_sm")


In [50]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []} 
names = []
phones = []
emails = []
skills = []

In [51]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile(
        "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    )
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [52]:
for file in os.listdir('/content/resume2/CV/'):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join('/content/resume2/CV',file))
        parse_content(txt)

Reading.....Alisson ParkerCV.pdf
/content/resume2/output/txt/Alisson ParkerCV.txt saved successfully!!!
Alisson Parker-Wright
alli1414parks@mail.com
Extraction completed successfully!!!
Reading.....AshleyMilesCV.pdf
/content/resume2/output/txt/AshleyMilesCV.txt saved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!
Reading.....John DominicCV.pdf
/content/resume2/output/txt/John DominicCV.txt saved successfully!!!
John Dominic
johndominic@mail.com
Extraction completed successfully!!!


In [53]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills


In [54]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright,['8569878511'],alli1414parks@mail.com,"{'python', 'tableau', 'java'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'sql', 'tableau'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'python', 'hadoop', 'java'}"


In [56]:
result_df.to_csv('/content/resume2/output/csv/parsed_resumes.csv')